# AWS BEDROCK API EXAMPLE

In [1]:
import boto3
from botocore.exceptions import ClientError
import os
import json
import logging

from bedrock import BedrockWrapper
from dotenv import load_dotenv, find_dotenv

logger = logging.getLogger(__name__)

load_dotenv(find_dotenv())
aws_access_key_id = os.environ.get("aws_access_key_id")
aws_secret_access_key = os.environ.get("aws_secret_access_key")

### Generative AI with large language models available on AWS

In [9]:
def generate(prompt: str) -> str:
    boto3_bedrock = boto3.client(
        'bedrock-runtime',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1")

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 300})

    modelId = 'anthropic.claude-instant-v1'
    contentType = 'application/json'

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    return response_body['completion']

response = generate("Human:2+2=? Assistant:")
print(response)

 4


#### Multimodal LLM

In [17]:
import base64

def generate_multimodal(image_path: str):
    boto3_bedrock = boto3.client(
        'bedrock-runtime',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1")
    image_base64 = base64.b64encode(open(image_path, 'rb').read()).decode('utf-8')
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": image_base64
                            }
                        },
                        {
                            "type": "text",
                            "text": "What's in this image?"
                        }
                    ]
                }
            ]
        }
    )

    modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
    contentType = 'application/json'

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    return response_body

response = generate_multimodal('picture.jpg')

print(response)

{'id': 'msg_bdrk_01CMVx7ebAQMW9PuevC34C6J', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': "This image shows a close-up portrait of a brown cow. The cow has a distinctive white marking down the center of its face, contrasting with its reddish-brown fur. Its eyes are clearly visible, giving it an alert and curious expression. The cow's ears are tagged with yellow identification tags, visible on both sides. One tag shows the number 5848, indicating this is likely a farm animal being tracked. The cow has a somewhat shaggy, fluffy appearance, especially noticeable in the tuft of hair on top of its head. The background is blurred, but appears to be an outdoor setting, possibly a farm or pasture. This image captures the cow's individual character and the details of its facial features very well."}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 385, 'output_tokens': 158}}


### Get model details via API

In [3]:
def usage_demo():
    """
    Shows how to list the available foundation models.
    This demonstration gets the list of available foundation models and
    prints their respective summaries.
    """
    logging.basicConfig(level=logging.INFO)
    print("-" * 88)
    print("Welcome to the Amazon Bedrock demo.")
    print("-" * 88)

    bedrock_client = boto3.client(
        service_name="bedrock",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1"
    )

    wrapper = BedrockWrapper(bedrock_client)

    print("Listing the available foundation models.")

    try:
        for model in wrapper.list_foundation_models():
            print_model_details(model)
    except ClientError:
        logger.exception("Couldn't list foundation models.")
        raise

    print("Getting the details of an individual foundation model.")

    model_id = "amazon.titan-embed-text-v1"

    try:
        print_model_details(wrapper.get_foundation_model(model_id))
    except ClientError:
        logger.exception(f"Couldn't get foundation model {model_id}.")
        raise


def print_model_details(model):
    print("\n" + "=" * 42)
    print(f' Model: {model["modelId"]}')
    print("-" * 42)
    print(f' Name: {model["modelName"]}')
    print(f' Provider: {model["providerName"]}')
    print(f' Model ARN: {model["modelArn"]}')
    print(f' Lifecycle status: {model["modelLifecycle"]["status"]}')
    print(f' Input modalities: {model["inputModalities"]}')
    print(f' Output modalities: {model["outputModalities"]}')
    print(f' Supported customizations: {model["customizationsSupported"]}')
    print(f' Supported inference types: {model["inferenceTypesSupported"]}')
    if "responseStreamingSupported" in model:
        print(f' Response streaming supported: {model["responseStreamingSupported"]}')

    print("=" * 42)

usage_demo()

----------------------------------------------------------------------------------------
Welcome to the Amazon Bedrock demo.
----------------------------------------------------------------------------------------
Listing the available foundation models.


INFO:bedrock:Got 26 foundation models.



 Model: amazon.titan-text-express-v1:0:8k
------------------------------------------
 Name: Titan Text G1 - Express
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-model/amazon.titan-text-express-v1:0:8k
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT']
 Output modalities: ['TEXT']
 Supported customizations: []
 Supported inference types: []
 Response streaming supported: True

 Model: amazon.titan-text-express-v1
------------------------------------------
 Name: Titan Text G1 - Express
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-model/amazon.titan-text-express-v1
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT']
 Output modalities: ['TEXT']
 Supported customizations: []
 Supported inference types: ['ON_DEMAND']
 Response streaming supported: True

 Model: amazon.titan-text-lite-v1:0:4k
------------------------------------------
 Name: Titan Text G1 - Lite
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-m